# Model testing

Evaluate the saved XGBoost regressor on the held-out test dataset.

In [28]:
import joblib
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


In [29]:
final_model = joblib.load("xgb_reg_movie_score_cv_no_validation.joblib")
df_test = pd.read_csv(r"../data/test_dataset.csv", sep=";")


In [34]:
target_col = "movie_score"
drop_cols = ["movie_score", "averageRating", "numVotes", "Unnamed: 0", "_orig_order"]

In [35]:
X_test = df_test.drop(columns = drop_cols)

In [ ]:
# Load model and data
y_true = df_test[target_col].values if target_col in df_test.columns else None
X_test = df_test.drop(columns=[c for c in drop_cols if c in df_test.columns])

print(f"Loaded test set with shape {df_test.shape}; features shape {X_test.shape}")


In [ ]:
# Predict
y_pred = final_model.predict(X_test)
print("Predictions generated.")


In [ ]:
# Metrics (only if ground truth available)
if y_true is not None:
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    print(f"MAE:  {mae:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"R2:   {r2:.4f}")
else:
    print("Ground-truth 'movie_score' not found in test set; only predictions are available.")


In [ ]:
y_true.mean()

In [ ]:
# Preview predictions
preview = pd.DataFrame({"predicted_movie_score": y_pred})
if y_true is not None:
    preview.insert(0, "actual_movie_score", y_true)


In [48]:
importances = final_model.feature_importances_

# Feature names (if trained with a DataFrame)
feature_names = final_model.feature_names_in_

# Create sorted DataFrame
top_10_features = (
    pd.DataFrame({
        "feature": feature_names,
        "importance": importances
    })
    .sort_values("importance", ascending=False)
    .head(10)
)